# Retrieve the results from a thread: `future` & `promise`

C ++ 11 offers a few ways to retrieve the output of a thread, without having to go through shared data and locks (apparently). This simpler style of programming is referred to as "asynchronous programming".

## Using the function `async`

If you do not want to interfere in some thread during its progress, but simply launch its execution, do something else in parallel and then wait and use thread's result: use a call to `std::async` instead.

The call to **`std::async`** is **non-blocking**, and immediatly returns a variable of type **`std::future <T>`**. Only later on, when one consult the value of this variable thanks to a call to **`get()`**, then the program will **block** until the result is available.

In [1]:
%%file tmp.async.h

#include <cstdio>
#include <chrono>
#include <thread>
#include <future>
#include <cassert>

using namespace std::chrono_literals ;

Writing tmp.async.h


In [2]:
%%file tmp.async-add.h

int addition( int nb )
 {
  int res = 0 ;
  for ( int i=1 ; i<=nb ; ++i )
   {
    std::this_thread::sleep_for(10us) ;
    int sum = i+i ;
    printf("...addition : %d => %d\n",i,sum) ;
    res += i ;
   }
  return res ;
 }

Writing tmp.async-add.h


In [3]:
%%file tmp.async-mul.h

int multiplication( int nb )
 {
  int res = 1 ;
  for ( int i=1 ; i<=nb ; ++i )
   {
    std::this_thread::sleep_for(50us) ;
    int square = i*i ;
    printf("...multiplication : %d => %d\n",i,square) ;
    res *= i ;
   }
  return res ;
 }

Writing tmp.async-mul.h


In [4]:
%%file tmp.async.cpp

#include "tmp.async.h"
#include "tmp.async-add.h"
#include "tmp.async-mul.h"

int main( int argc, char * argv[] )
 {
  assert(argc==2) ;
  int nb = atoi(argv[1]) ;
  std::future<int> res1 = std::async(addition,nb) ;
  std::future<int> res2 = std::async(multiplication,nb) ;
  //...
  printf("=> final addition: %d\n",res1.get()) ;
  printf("=> final multiplication: %d\n",res2.get()) ;
  return 0 ;
 }

Writing tmp.async.cpp


In [5]:
%%file tmp.async.sh
echo

rm -f tmp.async.exe \
&& g++ -std=c++17 -pthread tmp.async.cpp -o tmp.async.exe\
&& ./tmp.async.exe $*

echo

Writing tmp.async.sh


In [6]:
! bash -l tmp.async.sh 5


...addition : 1 => 2
...multiplication : 1 => 1
...addition : 2 => 4
...addition : 3 => 6
...multiplication : 2 => 4
...addition : 4 => 8
...multiplication : 3 => 9
...addition : 5 => 10
=> final addition: 15
...multiplication : 4 => 16
...multiplication : 5 => 25
=> final multiplication: 120



## Making promises

More generally, if we want to have a little more control over the course of the underlying thread, we can organize the recovery of a result by connecting one or more objects of type `future` in the main client code, together with one or several objects of type `promise` in to the supplier thread.

The same function can make several promises. In addition, it may still have things to do after the results are made available: the blocking call to `get()` on all expected values ​​does not mean that the threads have finished what they have to do. Unlike using `std::async`, you have to add explicit calls to `join()` on all threads again, or they will be finished cleanly.

In [7]:
%%file tmp.async-add.h

void addition( int nb, std::promise<int> prom )
 {
  int res = 0 ;
  for ( int i=1 ; i<=nb ; ++i )
   {
    res += i ;
    std::this_thread::sleep_for(10us) ;
    printf("...addition : %d => %d\n",i,res) ;
   }
  prom.set_value(res) ;
  std::this_thread::sleep_for(100us) ;
  printf("...addition cleaning\n") ;
 }

Overwriting tmp.async-add.h


In [8]:
%%file tmp.async-mul.h

void multiplication( int nb, std::promise<int> prom )
 {
  int res = 1 ;
  for ( int i=1 ; i<=nb ; ++i )
   {
    res *= i ;
    std::this_thread::sleep_for(50us) ;
    printf("...multiplication : %d => %d\n",i,res) ;
   }
  prom.set_value(res) ;
  std::this_thread::sleep_for(100us) ;
  printf("...multiplication cleaning\n") ;
 }

Overwriting tmp.async-mul.h


In [9]:
%%file tmp.async.cpp

#include "tmp.async.h"
#include "tmp.async-add.h"
#include "tmp.async-mul.h"

int main( int argc, char * argv[] )
 {
  assert(argc==2) ;
  int nb = atoi(argv[1]) ;
     
  std::promise<int> res1_promise ;
  std::promise<int> res2_promise ;
     
  std::future<int> res1_future = res1_promise.get_future() ;
  std::future<int> res2_future = res2_promise.get_future() ;
     
  std::thread t1(addition,nb,std::move(res1_promise)) ;
  std::thread t2(multiplication,nb,std::move(res2_promise)) ;
                 
  //...
                 
  printf("=> global addition: %d\n",res1_future.get()) ;
  printf("=> global multiplication: %d\n",res2_future.get()) ;
     
  t1.join() ;
  t2.join() ;
 }

Overwriting tmp.async.cpp


In [10]:
! bash -l tmp.async.sh 5


...addition : 1 => 1
...multiplication : 1 => 1
...addition : 2 => 3
...multiplication : 2 => 2
...addition : 3 => 6
...addition : 4 => 10
...multiplication : 3 => 6
...addition : 5 => 15
=> global addition: 15
...multiplication : 4 => 24
...addition cleaning
...multiplication : 5 => 120
=> global multiplication: 120
...multiplication cleaning



## Our shared future

Like an instance of `std::thread`, an instance of `std::future` is non-copiable, only movable. If we want to have several threads waiting for the same input asynchronous computation, we will instead use an instance of `std::shared_future`, which is copiable. Each client thread will then have its own copy of the result.

Below, we make the two threads wait together for the availability of the value of `nb` before actually starting their calculations. No longer taking advantage of the `multiplication` launch delay, `addition` takes a longer time to start.

In [11]:
%%file tmp.async-add.h

int addition( std::shared_future<int> nb_future )
 {
  int res = 0 ;
  int nb = nb_future.get() ;
  for ( int i=1 ; i<=nb ; ++i )
   {
    std::this_thread::sleep_for(10us) ;
    int sum = i+i ;
    printf("...addition : %d => %d\n",i,sum) ;
    res += i ;
   }
  return res ;
 }

Overwriting tmp.async-add.h


In [12]:
%%file tmp.async-mul.h

int multiplication( std::shared_future<int> nb_future )
 {
  int res = 1 ;
  int nb = nb_future.get() ;
  for ( int i=1 ; i<=nb ; ++i )
   {
    std::this_thread::sleep_for(50us) ;
    int square = i*i ;
    printf("...multiplication : %d => %d\n",i,square) ;
    res *= i ;
   }
  return res ;
 }

Overwriting tmp.async-mul.h


In [13]:
%%file tmp.async.cpp

#include "tmp.async.h"
#include "tmp.async-add.h"
#include "tmp.async-mul.h"

int main( int argc, char * argv[] )
 {
  assert(argc==2) ;
  int nb = atoi(argv[1]) ;
     
  std::promise<int> nb_promise ;
  std::shared_future<int> nb_future(nb_promise.get_future()) ;
     
  std::future<int> res1 = std::async(addition,nb_future) ;
  std::future<int> res2 = std::async(multiplication,nb_future) ;
     
  nb_promise.set_value(nb) ;
     
  //...
     
  printf("=> global addition: %d\n",res1.get()) ;
  printf("=> global multiplication: %d\n",res2.get()) ;
     
  return 0 ;
 }

Overwriting tmp.async.cpp


In [14]:
! bash -l tmp.async.sh 5


...addition : 1 => 2
...multiplication : 1 => 1
...addition : 2 => 4
...addition : 3 => 6
...multiplication : 2 => 4
...addition : 4 => 8
...multiplication : 3 => 9
...addition : 5 => 10
=> global addition: 15
...multiplication : 4 => 16
...multiplication : 5 => 25
=> global multiplication: 120



# Questions ?

# Exercise

Modify the program below to use the `std::async()` function instead of the explicit threads.

We have already modified the function `complexes_pow`, so that it returns its slice of results, instead of storing it into an arry received by reference. But now **it does not compile** (on purpose) : you still have to upgrade the section *compute* in the main program.

Check your results with the usual commands and parameters.

In [ ]:
%%file tmp.async.cpp

#include <complex>
#include <vector>
#include <iostream>
#include <cassert>
#include <cmath>
#include <thread>

using Real = double ;
using Complex = std::complex<Real> ;
using Complexes = std::vector<Complex> ;

// random unitary complexes
void generate( Complexes & cs )
 {
  srand(1) ;
  for ( auto & c : cs )
   { 
    Real angle {rand()/(Real(RAND_MAX)+1)*2.0*M_PI} ;
    c = Complex{std::cos(angle),std::sin(angle)} ;
   }
 }

// compute a slice of xs^degree and return it
Complexes complexes_pow
 ( std::size_t num_slice, std::size_t nb_slices,
   Complexes const & xs, int degree )
 {
  assert((xs.size()%nb_slices)==0) ;  
  auto slice_size {xs.size()/nb_slices} ;
  auto min {num_slice*slice_size} ;
  Complexes ys(slice_size) ;
  for ( decltype(slice_size) i {0} ; i<slice_size ; ++i )
   {
    ys[i] = Complex{1.,0.} ;
    for ( int d=0 ; d<degree ; ++d )
     { ys[i] *= xs[i+min] ; }
   }
  return ys ;
 }

// display the angle of the global product
void postprocess( Complexes const & cs )
 {
  Complex prod {1.,0.} ;
  for( auto c : cs ) { prod *= c ; }
  double angle {atan2(prod.imag(),prod.real())} ;
  std::cout<<"result = "<<static_cast<int>(angle/2./M_PI*360.)<<"\n" ;
 }

// main program
int main ( int argc, char * argv[] )
 {
  assert(argc==4) ;
  std::size_t nbtasks {std::stoul(argv[1])} ;
  std::size_t dim {std::stoul(argv[2])} ;
  int degree {std::stoi(argv[3])} ;

  // prepare input
  Complexes input(dim) ;
  generate(input) ;
   
  // compute
  Complexes output(dim) ;
  std::size_t numtask ;
  std::vector<> workers ;
  for ( numtask = 0 ; numtask<nbtasks ; ++numtask )
   { workers.emplace_back(complexes_pow,numtask,nbtasks,std::ref(input),degree,std::ref(output)) ; }
  for ( auto & worker : workers )
   { worker.join() ; }
  
  // post-process
  postprocess(output) ;
 }

In [ ]:
%%file tmp.async.sh
echo

rm -f tmp.async.exe \
&& g++ -std=c++17 -lpthread tmp.async.cpp -o tmp.async.exe\
&& time ./tmp.async.exe $*

echo

In [ ]:
! bash -l tmp.async.sh 2 2 3

In [ ]:
! bash -l tmp.async.sh 4 1024 100000

© *CNRS 2020*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*